# S02: データ変換（HTML → Parquet）

HTMLをパースしてParquet形式で保存し、GitHubにプッシュすることでオンラインから参照可能にする

**実行環境**: Sagemaker

---
## 1. 環境設定

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルート
PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# データパス
HTML_DIR = PROJECT_ROOT / 'data' / 'raceHTML'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'

# ディレクトリ作成
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# ファイルパス
RACES_PARQUET = PROCESSED_DIR / 'races.parquet'
RESULTS_PARQUET = PROCESSED_DIR / 'results.parquet'

# GitHub RAW URL（プッシュ後にアクセス可能）
GITHUB_BASE = "https://raw.githubusercontent.com/iinumac/keiba_prediction/main/data/processed"

print(f"HTML_DIR: {HTML_DIR.resolve()}")
print(f"RACES_PARQUET: {RACES_PARQUET}")
print(f"RESULTS_PARQUET: {RESULTS_PARQUET}")

In [ ]:
# モジュールインポート
from scraper.parser import parse_multiple_html_full, classify_race_level

print("✅ モジュール読み込み完了")

---
## 2. 全HTMLをパース → Parquet保存

In [ ]:
def progress_callback(processed, total, skipped=0):
    pct = processed / total * 100 if total > 0 else 0
    print(f"\r進捗: {processed:,}/{total:,} ({pct:.1f}%)", end='')

print("📊 HTMLパースを開始...")
print(f"対象: {HTML_DIR}")

race_list, horse_list = parse_multiple_html_full(
    HTML_DIR,
    years=None,
    progress_callback=progress_callback
)

print(f"\n\n✅ パース完了")
print(f"   レース数: {len(race_list):,}")
print(f"   出走馬データ数: {len(horse_list):,}")

In [ ]:
# DataFrameに変換
races_df = pd.DataFrame(race_list)
results_df = pd.DataFrame(horse_list)

print(f"races_df: {races_df.shape}")
print(f"results_df: {results_df.shape}")

In [ ]:
# Parquet形式で保存
races_df.to_parquet(RACES_PARQUET, index=False, compression='snappy')
results_df.to_parquet(RESULTS_PARQUET, index=False, compression='snappy')

print("✅ Parquet保存完了")
print(f"   races.parquet: {RACES_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")
print(f"   results.parquet: {RESULTS_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")

---
## 3. データ確認

In [ ]:
# 統計
print("=== データ統計 ===")
print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")

if 'year' in races_df.columns:
    print("\n=== 年別レース数 ===")
    print(races_df.groupby('year').size().to_string())

In [ ]:
# レースレベル分布
if 'race_level' in races_df.columns:
    print("=== レースレベル分布 ===")
    print(races_df['race_level'].value_counts().sort_index().to_string())

In [ ]:
# サンプル表示
print("=== レースデータ サンプル ===")
display(races_df.tail(5))

In [ ]:
print("=== 出走馬データ サンプル ===")
display(results_df.tail(5))

---
## 4. GitHubにプッシュ

Parquetファイル生成後、以下のコマンドでプッシュ:

```bash
git add data/processed/*.parquet
git commit -m "Update race data parquet files"
git push
```

In [ ]:
# Gitコマンド実行（オプション）
import subprocess

os.chdir(PROJECT_ROOT)

# Parquetファイルを追加・コミット・プッシュ
# subprocess.run(['git', 'add', 'data/processed/*.parquet'], check=True)
# subprocess.run(['git', 'commit', '-m', 'Update race data parquet files'], check=True)
# subprocess.run(['git', 'push'], check=True)

print("⚠️ 手動でgit pushしてください（上のコマンドのコメントを外して実行も可能）")

---
## 5. オンラインからの読み込み方法

GitHubにプッシュ後、以下のコードで読み込み可能:

```python
import pandas as pd

# GitHub RAW URLから直接読み込み
GITHUB_BASE = "https://raw.githubusercontent.com/iinumac/keiba_prediction/main/data/processed"

races_df = pd.read_parquet(f"{GITHUB_BASE}/races.parquet")
results_df = pd.read_parquet(f"{GITHUB_BASE}/results.parquet")

# 特定の馬を検索
horse_id = "2022110077"
horse_results = results_df[results_df['horse_id'] == horse_id]
print(horse_results)
```

In [ ]:
# 特定の馬を検索するサンプル
horse_id = "2022110077"  # イリフィ
horse_results = results_df[results_df['horse_id'] == horse_id]

print(f"=== 馬ID: {horse_id} の成績 ===")
print(f"出走回数: {len(horse_results)}")
display(horse_results[['race_date', 'venue_name', 'distance', 'surface', 'finish_position', 'popularity', 'odds', 'prize_money']])